# ABD2

Wij gaan tijdens deze opdracht een food-collector simulatie bouwen.
In deze simulatie is het doel van de agents om zo veel mogelijk groen eten op te eten, terwijl ze rood eten vermijden.
We gaan deze simulatie grid-based maken. 

We maken gebruik van de gymnasium API: https://gymnasium.farama.org/

Ter visualisatie gebruiken we pygame.



### Simulation properties:
We beginnen in principe met een grid van 128x128, maar dit kan uitgebreid worden. 
We zullen de implementatie zoveel mogelijk scalable en aanpasbaar implementeren, door hyperparameters aan te maken voor alle belangrijke properties.

- Number of agents
- Grid size (default 128x128)
- Good food to total square ratio
- Good food spawning pattern (maybe)
- Bad food to total square ratio
- Bad food spawning pattern (maybe)

### Agent properties:
Actions:
- up
- down
- left
- right
- wait

Perception: full information (knows coordinates of other agents, good food, and bad food)

Reward function (not yet specified)

### Intelligence

De intelligentie komt hem hier voort uit het meest optimaal verzamelen van voedsel, en zodra er meerdere agents zijn ontstaat er ook een element van competitie wat zal leiden tot verschillende strategiën.



In [5]:
import numpy as np
import pygame
import gymnasium as gym
from gymnasium import spaces

In [4]:
env = gym.make("LunarLander-v2", render_mode="human")
observation, info = env.reset()

for _ in range(1000):
    action = env.action_space.sample()  # agent policy that uses the observation and info
    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        observation, info = env.reset()

env.close()